In [10]:
import os

import numpy as np
import pandas as pd

import pgd_attack
import run_attack

In [13]:
def to_df(arr):
    df = pd.DataFrame(arr)
    df.index.name = "Original Class"
    df.columns.name = "Target Class"
    
    return df

#### Naturally Trained

In config.json, set:  
"model_dir": "models/natural"  
"store_adv_path": "attacks/attack_nat.npy"  
"class_attack": true

In [ ]:
# Generate attack files
if not os.path.exists('attacks'):
    os.makedirs('attacks')

pgd_attack.main()

In [6]:
%%capture
# Obtain confusion matrix
nat_conf_mat = run_attack.main()

INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900
INFO:tensorflow:Restoring parameters from models/natural/checkpoint-24900


In [14]:
to_df(nat_conf_mat)

Target Class,0,1,2,3,4,5,6,7,8,9
Original Class,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


#### Adversarially Trained

In config.json, set:  
"model_dir": "models/adv_trained"  
"store_adv_path": "attacks/attack_adv.npy"  
"class_attack": true

In [ ]:
# Generate attack files
pgd_attack.main()

In [8]:
%%capture
# Obtain confusion matrix
adv_conf_mat = run_attack.main()

INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900
INFO:tensorflow:Restoring parameters from models/adv_trained/checkpoint-99900


In [15]:
to_df(adv_conf_mat)

Target Class,0,1,2,3,4,5,6,7,8,9
Original Class,,,,,,,,,,
0,1.000000,0.001020,0.002041,0.001020,0.001020,0.003061,0.007143,0.003061,0.003061,0.001020
1,0.000881,1.000000,0.007048,0.002643,0.005286,0.001762,0.006167,0.001762,0.010573,0.000881
2,0.008721,0.004845,0.997093,0.007752,0.002907,0.000000,0.001938,0.013566,0.013566,0.000000
3,0.002970,0.000990,0.012871,1.000000,0.000990,0.030693,0.000000,0.009901,0.030693,0.011881
4,0.002037,0.004073,0.006110,0.001018,0.995927,0.000000,0.010183,0.008147,0.003055,0.053971
5,0.007848,0.002242,0.002242,0.048206,0.003363,0.996637,0.024664,0.003363,0.019058,0.014574
6,0.011482,0.008351,0.002088,0.001044,0.009395,0.009395,0.994781,0.000000,0.008351,0.001044
7,0.000973,0.007782,0.014591,0.010700,0.011673,0.000973,0.000000,0.990272,0.003891,0.031128
8,0.022587,0.004107,0.013347,0.032854,0.013347,0.018480,0.012320,0.013347,0.997947,0.018480
